In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from ipynb.fs.full.IntradayRange import IntradayRange
#from ipynb.fs.full.IntradayRangeBreak import IntradayRangeBreak
from ipynb.fs.full.SMA import SimpleMovingAverage


In [4]:
df = pd.read_csv('Tradedata/W210708.csv')
df = df.dropna()
df = df.fillna(0)
#print(df)
df = df.sort_values(by=['Date/Time', 'Ticker'])
df = df.reset_index()  # make sure indexes pair with number of rows
prevtimestamp = df.at[2,'Date/Time']
allvalues = []
alltrades = []
validity = False
for index, row in df.iterrows():
    if index > 100000:
        break
    action = None
    if (row['Date/Time'] != prevtimestamp):
        currentTime = datetime.strptime(prevtimestamp, '%d-%m-%Y %H:%M:%S')
        if(row['Ticker'] == "BANKNIFTY"):
            validity = True
        if validity:
            #action = IntradayRange(currentTime.timetuple().tm_yday, (currentTime.timestamp()% 86400 -13500)/60 , allvalues)
            action = SimpleMovingAverage(currentTime.timetuple().tm_yday, (currentTime.timestamp()% 86400 -13500)/60 , allvalues)
        #IntradayRangeBreak(1,1,allvalues)
        allvalues = []
        validity = False
    
    allvalues.append([row['Ticker'] ,row['Close'], row['Volume'], row['Open Interest']])
    if (action is not None):
        for trade in action:
            trade.append(prevtimestamp)
            alltrades.append(trade)
    prevtimestamp = row['Date/Time']

#print(alltrades)

KeyError: 5666346346342

In [ ]:


def findTrade(isPE, premium, data):
    if isPE:
        filtereddata = data.loc[data[0].str.match('^BANKNIFTY.*PE.*')== True]
    else:
        filtereddata = data.loc[data[0].str.match('^BANKNIFTY.*CE.*')== True]
    filtereddata = filtereddata.sort_values(by=[1])
    filtereddata = filtereddata.loc[(data[1]>premium)== True][:1]
    return [filtereddata[0].values[0],filtereddata[1].values[0]]


# trade format: [name, price, buy/sell, strategy]
#sl,target in portion of 1 eg, sl:0.2 (20%), target: 0.5 (50%)
def exitTrades(trades, currdata, sl, target):
    response = []
    currdf = pd.DataFrame(currdata)
    curr = currdf.loc[currdata[0] == "BANKNIFTY"][1].iloc[0]
    print(currdf)
    for trade in trades:
        if((trade[2] == "BUY") and ((curr > (trade[1]*(1+target))) or (curr < (trade[1]*(1-sl))))):
            tradeprice = currdata.loc[currdata[0] == trade[0]][1].iloc[0]
            newtrade = trade.copy()
            newtrade[1] = tradeprice
            newtrade[2] = "SELL"
            response.append(newtrade)
        elif ((trade[2] == "SELL") and ((curr > (trade[1]*(1+sl))) or (curr < (trade[1]*(1-target))))):
            tradeprice = currdata.loc[currdata[0] == trade[0]][1].iloc[0]
            newtrade = trade.copy()
            newtrade[1] = tradeprice
            newtrade[2] = "BUY"
            response.append(newtrade)
    if(len(response) > 0):
        print(curr,sl,target)
        print("11111\n")
        print(trades)
        print("22222\n")
        print(response)
        print("33333\n")
        
    return response
              